<h3 style="text-align: center; font-family: Arial, sans-serif; color: #4CAF50;">TSS - Temporal Similarity Search : POC complete DATASET WITH API - avec enregistrement fichier .json volumineux et fichier parquet petit (sur le local pas de API) mais trop lourd dans la recherche victorielle</h3>
<ul style="font-family: Arial, sans-serif; font-size: 12pt; color: #333;">
</ul>

In [ ]:
pip install dask

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import json
import time
from tqdm import tqdm
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [69]:
# ==================================================
# 1. Chargement et préparation des données avec Dask
# ==================================================

# Charger les données en Dask DataFrame
df_dask = dd.read_csv("../Datasources/MetroPT3_imputed_final.csv", delimiter=",", decimal=".", parse_dates=['timestamp'])

# Sélection des colonnes continues et catégoriques
continuous_features  = ["TP2", "DV_pressure", "Oil_temperature", "Motor_current", "Reservoirs"]
categorical_features = ["COMP", "DV_eletric", "Towers", "LPS", "Pressure_switch", "Oil_level", "Caudal_impulses"]
columns_to_keep = ["timestamp", "panne"] + continuous_features + categorical_features

# Filtrer les colonnes utiles
df_dask = df_dask[columns_to_keep]

print("Données chargées avec succes.")

Données chargées avec succes.


In [3]:
# =================================================
# 2. Séparation des données en entrainement et test
# =================================================
###################################################################################################
################ Train : Panne1 + Panne2 + Panne3                                      ############
################ Test  : Panne4                                                        ############
###################################################################################################
################ Classe 0 : Pas de panne détectée                                      ############
################ Classe 1 : En plein panne                                             ############                              
################ Classe 2 : Panne prévue dans moins de 15 minutes                      ############ 
###################################################################################################
################ 2020-04-17 23:30:00 -- 2020-04-17 23:44:50 : 0                        ############
################ 2020-04-17 23:45:00 -- 2020-04-17 23:59:50 : 2                        ############
################ 2020-04-18 00:00:00 -- 2020-04-18 00:15:00 : 1                        ############
################ 2020-04-18 00:15:10 -- 2020-04-18 00:30:00 : 0                        ############
################ 2020-04-18 00:30:10 -- 2020-04-18 00:45:00 : 2                        ############
################ 2020-04-18 00:45:10 -- 2020-04-18 01:00:10 : 1                        ############
################ 2020-04-18 01:00:20 -- 2020-04-18 01:15:10 : 0                        ############
################ 2020-04-18 01:15:20 -- 2020-04-18 01:30:10 : 2                        ############
################ 2020-04-18 01:30:20 -- 2020-04-18 01:45:20 : 1                        ############ 
################ 2020-04-18 01:45:30 -- 2020-04-18 02:00:20 : 0                        ############
################ 2020-04-18 02:00:30 -- 2020-04-18 02:15:20 : 2                        ############
################ 2020-04-18 02:15:30 -- 2020-04-18 02:30:30 : 1                        ############
###################################################################################################


# Liste des intervalles de pannes
pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]

# Définition des périodes d'entrainement
train_periods = [{'start': '2020-02-01 00:00:00', 'end': '2020-06-07 14:30:00'}]
test_periods  = [{'start': '2020-06-07 14:30:10', 'end': '2020-09-01 03:59:50'}]

# Séparer les données d'entrainement et de test
train_df = dd.concat([
    df_dask[(df_dask['timestamp'] >= period['start']) & (df_dask['timestamp'] <= period['end'])]
    for period in train_periods
])

test_df = dd.concat([
    df_dask[(df_dask['timestamp'] >= period['start']) & (df_dask['timestamp'] <= period['end'])]
    for period in test_periods
])

# Regrouper les partitions pour éviter les coupures temporelles
train_df = train_df.repartition(npartitions=4).reset_index(drop=True)
test_df = test_df.repartition(npartitions=1).reset_index(drop=True)

# Calcul du nombre de lignes
print(f"Entrainement : {train_df.shape[0].compute()} lignes")
print(f"Test : {test_df.shape[0].compute()} lignes")


Entrainement : 1102501 lignes
Test : 739259 lignes


In [4]:
# ===============================================
# 3. Génération des fenêtres glissantes avec Dask
# ===============================================

window_size = 6  # Taille de la fenêtre
step_size = 1      # Pas de glissement

def create_windows(df):
    windows = []
    df = df.compute()  # Convertir en Pandas pour traitement en mémoire
    for i in tqdm(range(0, len(df) - window_size + 1, step_size), desc="Création des fenêtres"):
        window_data = df.iloc[i:i + window_size][continuous_features].values.flatten()
        panne_value = df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
        windows.append({
            "index": i,
            "timestamp": df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist(),
            "panne": panne_value
        })
    return windows

# Génération des fenêtres d'entrainement et de test
print("Génération des fenêtres d'entrainement...")
train_windows = create_windows(train_df)

print("Génération des fenêtres de test...")
test_windows = create_windows(test_df)

print(f"Nombre de fenêtres d'entrainement : {len(train_windows)}")
print(f"Nombre de fenêtres de test : {len(test_windows)}")


Génération des fenêtres d'entrainement...


Création des fenêtres: 100%|████████████████████████████████████████████████| 1102496/1102496 [21:26<00:00, 856.79it/s]


Génération des fenêtres de test...


Création des fenêtres: 100%|██████████████████████████████████████████████████| 739254/739254 [14:27<00:00, 852.39it/s]

Nombre de fenêtres d'entrainement : 1102496
Nombre de fenêtres de test : 739254


In [5]:
# Afficher le premier vecteur d'entrainement
first_train_vector = train_windows[0]["sensor_data"]
print("Premier vecteur d'entrainement :")
print(first_train_vector)

Premier vecteur d'entrainement :
[-0.0120000000000004, -0.0240000000000009, 53.60000000000001, 0.04, 9.358, -0.0139999999999993, -0.0219999999999984, 53.67500000000001, 0.04, 9.348, -0.0120000000000004, -0.0219999999999984, 53.60000000000001, 0.0424999999999995, 9.338, -0.0120000000000004, -0.0219999999999984, 53.42499999999998, 0.04, 9.328, -0.0120000000000004, -0.0219999999999984, 53.47500000000001, 0.04, 9.318, -0.0120000000000004, -0.0240000000000009, 53.49999999999999, 0.04, 9.308]


In [7]:
# ==============================================================================
# 4. Sauvegarde des fenêtres en format Parquet pour utilisation future avec Dask
# ==============================================================================

import pandas as pd
import dask.dataframe as dd
import json
import time

# Démarrer le chronometre
start_time = time.time()

# Convertir les données en JSON string pour les sauvegarder correctement dans Parquet
train_df_windows = pd.DataFrame(train_windows)
train_df_windows["sensor_data"] = train_df_windows["sensor_data"].apply(json.dumps)

# Convertir en DataFrame Dask avec plusieurs partitions
train_df_dask = dd.from_pandas(train_df_windows, npartitions=4)

# Sauvegarde en Parquet sans l'argument index=False
train_df_dask.to_parquet("Generated_Files/Model_1_TSS_POC_DASK_train_vectors.parquet")

print("Vecteurs d'entrainement sauvegardés au format Parquet.")

# Arrêter le chronometre
end_time = time.time()

# Calculer le temps d'exécution
execution_time = end_time - start_time
print(f"Temps d'exécution: {execution_time:.2f} secondes")



Vecteurs d'entrainement sauvegardés au format Parquet.
Temps d'exécution: 41.91 secondes


In [8]:
# ==========================================================
# 5.  Chargement des vecteurs d'entrainement pour prédiction
# ==========================================================

import dask.dataframe as dd
import numpy as np
import json
from scipy.spatial.distance import cdist

# Démarrer le chronometre
start_time = time.time()

# Charger les données d'entrainement sauvegardées
train_df_loaded = dd.read_parquet("Generated_Files/Model_1_TSS_POC_DASK_train_vectors.parquet")

# Convertir la colonne 'sensor_data' de JSON string en liste pour la recherche vectorielle
train_df_loaded["sensor_data"] = train_df_loaded["sensor_data"].apply(json.loads, meta=("sensor_data", "object"))

# Transformation en format utilisable pour la recherche vectorielle
X_train_loaded = np.vstack(train_df_loaded["sensor_data"].compute())
y_train_loaded = train_df_loaded["panne"].compute().values

print(f"Données d'entrainement chargées : {X_train_loaded.shape}")

# Vérification du nombre de partitions
print(f"Nombre de partitions dans le DataFrame Dask : {train_df_loaded.npartitions}")


# Afficher un extrait des données chargées
display(train_df_loaded.head())

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

Données d'entrainement chargées : (1102496, 30)
Nombre de partitions dans le DataFrame Dask : 4


,index,timestamp,sensor_data,panne
0,0,2020-02-01 00:00:00,"[-0.0120000000000004, -0.0240000000000009, 53....",0
1,1,2020-02-01 00:00:10,"[-0.0139999999999993, -0.0219999999999984, 53....",0
2,2,2020-02-01 00:00:20,"[-0.0120000000000004, -0.0219999999999984, 53....",0
3,3,2020-02-01 00:00:30,"[-0.0120000000000004, -0.0219999999999984, 53....",0
4,4,2020-02-01 00:00:40,"[-0.0120000000000004, -0.0219999999999984, 53....",0


Temps d'exécution: 48.17 secondes


In [9]:
# Préparation du nouvel ensemble de test

test_df = dd.concat([
    df_dask[(df_dask['timestamp'] >= period['start']) & (df_dask['timestamp'] <= period['end'])]
    for period in test_periods
])

# Regrouper les partitions pour éviter les coupures temporelles
test_df = test_df.repartition(npartitions=1).reset_index(drop=True)

# Vérification du nombre de lignes et partitions
print(f"Nombre de lignes dans le test : {test_df.shape[0].compute()}")
print(f"Nombre de partitions dans le test : {test_df.npartitions}")

Nombre de lignes dans le test : 739259
Nombre de partitions dans le test : 1


In [11]:
# =======================================================================
# 6. Génération des fenêtres glissantes pour l'ensemble de test avec Dask
# =======================================================================

# Démarrer le chronometre
start_time = time.time()

# Fonction pour créer des fenêtres de test
window_size = 6    # Taille de la fenêtre
step_size = 1      # Pas de glissement

def create_windows(df):
    windows = []
    df = df.compute()  # Convertir en Pandas pour traitement en mémoire
    for i in range(0, len(df) - window_size + 1, step_size):
        window_data = df.iloc[i:i + window_size][continuous_features].values.flatten()
        panne_value = df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
        windows.append({
            "index": i,
            "timestamp": df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist(),
            "panne": panne_value
        })
    return windows

# Générer les fenêtres de test
print("Génération des fenêtres de test...")
test_windows = create_windows(test_df)

print(f"Nombre de fenêtres générées : {len(test_windows)}")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

Génération des fenêtres de test...
Nombre de fenêtres générées : 739254
Temps d'exécution: 831.23 secondes


In [12]:
# ====================================
# 7. Prédiction sur l'ensemble de test
# ====================================

# Démarrer le chronometre
start_time = time.time()

# Fonction de recherche des k plus proches voisins en utilisant la distance euclidienne
def find_similar_windows(test_vector, train_vectors, train_labels, k=5):
    # Calculer les distances entre le vecteur de test et les vecteurs d'entrainement
    distances = cdist([test_vector], train_vectors, metric="euclidean")
    
    # Trouver les indices des k plus proches voisins
    nearest_indices = np.argsort(distances, axis=1)[:, :k]

    # Prédiction par vote majoritaire
    similar_pannes = train_labels[nearest_indices.flatten()]
    predicted_panne = np.bincount(similar_pannes).argmax()

    return predicted_panne

# Appliquer la prédiction sur l'ensemble de test
predictions = []
for test_sample in tqdm(test_windows, desc="Prédictions en cours"):
    test_vector = test_sample["sensor_data"]
    predicted_panne = find_similar_windows(test_vector, X_train_loaded, y_train_loaded)
    
    predictions.append({
        "timestamp": test_sample["timestamp"],
        "actual_panne": test_sample["panne"],
        "predicted_panne": predicted_panne
    })

# Convertir en DataFrame Pandas puis en Dask
predictions_df = pd.DataFrame(predictions)
predictions_dask = dd.from_pandas(predictions_df, npartitions=1)

# Sauvegarder les prédictions en format Parquet
predictions_dask.to_parquet("Generated_Files/test_predictions.parquet")
print("Prédictions sauvegardées avec succes.")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")


Prédictions en cours: 100%|█████████████████████████████████████████████████| 739254/739254 [23:56:26<00:00,  8.58it/s]


Prédictions sauvegardées avec succes.
Temps d'exécution: 86198.52 secondes


In [56]:
import pandas as pd

# Charger le fichier CSV
df_predictions = pd.read_csv(
    "Generated_Files/final_predictions.csv",
    header=None,
    names=["timestamp", "panne_réelle", "panne_prédite"],
    dtype={"timestamp": str, "panne_réelle": str, "panne_prédite": str},  # Charger d'abord en str
    low_memory=False
)

# Convertir les colonnes en type numérique pour éliminer les incohérences
df_predictions["panne_réelle"] = pd.to_numeric(df_predictions["panne_réelle"], errors='coerce')
df_predictions["panne_prédite"] = pd.to_numeric(df_predictions["panne_prédite"], errors='coerce')

# Supprimer les valeurs nulles si nécessaire
df_predictions.dropna(inplace=True)

# Calcul des répartitions des classes réelles et prédites
class_counts_real = df_predictions["panne_réelle"].value_counts().sort_index()

# Ajouter une colonne de vérification pour identifier les prédictions correctes
df_predictions["correct"] = df_predictions["panne_réelle"] == df_predictions["panne_prédite"]

# Calcul du nombre de bonnes prédictions par classe
correct_predictions = df_predictions[df_predictions["correct"]].groupby("panne_réelle").size().reindex(class_counts_real.index, fill_value=0)

# Création du tableau récapitulatif sans la colonne "Prédictions"
recap_data_corrected = {
    "Classe": [
        f"{int(class_label)} (Pas de panne)" if class_label == 0 else
        f"{int(class_label)} (En pleine panne)" if class_label == 1 else
        f"{int(class_label)} (Panne imminente)"
        for class_label in class_counts_real.index
    ],
    "Observations réelles": class_counts_real.values,
    "Prédictions correctes": correct_predictions.values,
    "Remarque": [
        "" if correct_predictions[class_label] > 0 else "Le modele n'a jamais correctement prédit cette classe."
        for class_label in class_counts_real.index
    ]
}

# Création du DataFrame corrigé
recap_df_corrected = pd.DataFrame(recap_data_corrected)

# Affichage du tableau récapitulatif
recap_df_corrected


,Classe,Observations réelles,Prédictions correctes,Remarque
0,0 (Pas de panne),737454,737141,
1,1 (En pleine panne),1621,10,
2,2 (Panne imminente),179,0,Le modele n'a jamais correctement prédit cette...


In [13]:
# ========================================
# 8. Évaluation des performances du modele
# ========================================

# Charger les prédictions sauvegardées pour analyse
predictions_df = dd.read_parquet("Generated_Files/test_predictions.parquet").compute()

# Calcul des métriques de performance
y_true = predictions_df["actual_panne"].values
y_pred = predictions_df["predicted_panne"].values

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Afficher les résultats
print(f"Exactitude (Accuracy) : {accuracy:.2f}")
print(f"Précision : {precision:.2f}")
print(f"Rappel (Recall) : {recall:.2f}")
print(f"Score F1 : {f1:.2f}")

C:\Users\usermine\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Exactitude (Accuracy) : 1.00
Précision : 1.00
Rappel (Recall) : 1.00
Score F1 : 1.00


In [14]:
# Sauvegarde au format CSV pour analyse ultérieure
predictions_df.to_csv("Generated_Files/final_predictions.csv", index=False)

print("Les prédictions finales ont été enregistrées avec succes.")


Les prédictions finales ont été enregistrées avec succes.


In [24]:
#  1 : Vérification de la séparation des données d'entrainement et de test.

# Vérification des plages de timestamps
train_start = train_min_timestamp
train_end = train_max_timestamp
test_start = test_min_timestamp
test_end = test_max_timestamp

print(f"Période d'entrainement : {train_start} -> {train_end}")
print(f"Période de test        : {test_start} -> {test_end}")

# Vérification d'un chevauchement éventuel
if train_end >= test_start:
    print("Il y a un chevauchement entre l'entrainement et le test. Vérifiez la séparation des données.")
else:
    print("Pas de chevauchement détecté.")


Période d'entrainement : 2020-02-01 00:00:00 -> 2020-06-07 14:29:10
Période de test        : 2020-06-07 14:30:10 -> 2020-09-01 03:59:00
Pas de chevauchement détecté.


In [26]:
# 2 : Vérification des statistiques descriptives sur les données d'entrainement et de test

import numpy as np
import pandas as pd

# Extraction des valeurs des capteurs des fenêtres d'entrainement et de test
train_values = np.array([sample["sensor_data"] for sample in train_windows]).flatten()
test_values = np.array([sample["sensor_data"] for sample in test_windows]).flatten()

# Calcul des statistiques descriptives
def get_statistics(values, dataset_name):
    stats = {
        "Min": np.min(values),
        "Max": np.max(values),
        "Moyenne": np.mean(values),
        "Médiane": np.median(values),
        "Écart-type": np.std(values)
    }
    print(f"Statistiques pour {dataset_name}:")
    for key, value in stats.items():
        print(f"{key}: {value:.4f}")
    print("\n")

# Affichage des statistiques pour l'entrainement et le test
get_statistics(train_values, "Entrainement")
get_statistics(test_values, "Test")

# Ces différences peuvent expliquer pourquoi le modele obtient des performances parfaites. 
# Si les données de test ne sont pas bien représentées par celles d'entrainement (notamment pour les valeurs extrêmes),
# le modele pourrait ne pas être confronté à des cas difficiles pendant le test. 
# Cela peut indiquer un probleme potentiel de surajustement (overfitting).

Statistiques pour Entrainement:
Min: -0.0320
Max: 83.1250
Moyenne: 14.0246
Médiane: 1.4520
Écart-type: 22.9968


Statistiques pour Test:
Min: -0.0320
Max: 89.0500
Moyenne: 14.9613
Médiane: 3.1548
Écart-type: 24.6296




In [27]:
# 3 : Vérification de la distribution des classes
from collections import Counter

# Comptage des occurrences des classes dans l'entrainement et le test
train_classes = [sample["panne"] for sample in train_windows]
test_classes = [sample["panne"] for sample in test_windows]

# Affichage de la distribution des classes
print("Distribution des classes dans l'entrainement :", Counter(train_classes))
print("Distribution des classes dans le test :", Counter(test_classes))

Distribution des classes dans l'entrainement : Counter({0: 1072084, 1: 29875, 2: 537})
Distribution des classes dans le test : Counter({0: 737454, 1: 1621, 2: 179})


In [45]:
# 4 : Vérification de l'imputation des données
    
# Vérification des valeurs imputées dans l'entrainement et le test
train_missing = sum(np.isnan(train_values))
test_missing = sum(np.isnan(test_values))

print(f"Nombre de valeurs manquantes dans l'ensemble d'entrainement : {train_missing}")
print(f"Nombre de valeurs manquantes dans l'ensemble de test : {test_missing}")


Nombre de valeurs manquantes dans l'ensemble d'entrainement : 0
Nombre de valeurs manquantes dans l'ensemble de test : 0


In [29]:
# 5 : Corrélation entre les ensembles

# Convertir les données en DataFrame pour calculer les corrélations
train_df = pd.DataFrame([sample["sensor_data"] for sample in train_windows])
test_df = pd.DataFrame([sample["sensor_data"] for sample in test_windows])

# Calcul des corrélations
train_corr = train_df.corr()
test_corr = test_df.corr()

# Comparaison des corrélations (différence absolue)
corr_diff = (train_corr - test_corr).abs()

# Afficher les variables ayant des différences de corrélation élevées
print("Différences de corrélation entre l'entrainement et le test :")
display(corr_diff.sort_values(by=corr_diff.columns[0], ascending=False).head(10))


Différences de corrélation entre l'entrainement et le test :


,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
1,0.491452,0.000000,0.538979,0.318475,0.132623,0.521240,0.772223,0.533276,0.324220,0.128068,...,0.520987,0.774669,0.533786,0.324264,0.128468,0.522750,0.776426,0.534321,0.324946,0.129524
26,0.455603,0.776426,0.546232,0.302767,0.132693,0.451682,0.774669,0.542598,0.301035,0.130951,...,0.449399,0.772223,0.536552,0.299916,0.128415,0.491452,0.000000,0.538975,0.318474,0.132623
21,0.451682,0.774669,0.542599,0.301036,0.130951,0.450513,0.774082,0.540307,0.300416,0.130122,...,0.491452,0.000000,0.538976,0.318474,0.132623,0.521239,0.772223,0.533273,0.324219,0.128068
16,0.450513,0.774082,0.540307,0.300416,0.130122,0.448943,0.771949,0.538247,0.299827,0.129198,...,0.521239,0.772223,0.533274,0.324219,0.128068,0.520472,0.771950,0.532802,0.324035,0.128256
6,0.449399,0.772223,0.536554,0.299917,0.128415,0.491452,0.000000,0.538978,0.318475,0.132623,...,0.520517,0.774082,0.533543,0.324082,0.128343,0.520987,0.774669,0.533785,0.324264,0.128468
11,0.448943,0.771949,0.538248,0.299827,0.129198,0.449399,0.772223,0.536554,0.299916,0.128415,...,0.520472,0.771950,0.532803,0.324036,0.128256,0.520517,0.774082,0.533542,0.324082,0.128343
7,0.298339,0.533276,0.017492,0.212965,0.348765,0.289260,0.538978,0.000000,0.207572,0.368510,...,0.354179,0.540307,0.021187,0.242842,0.347118,0.366133,0.542598,0.024042,0.249422,0.345884
2,0.289261,0.538979,0.000000,0.207575,0.368513,0.327301,0.536554,0.017492,0.228454,0.348899,...,0.366133,0.542599,0.024041,0.249425,0.345886,0.372235,0.546232,0.024323,0.252845,0.347991
12,0.284328,0.532805,0.018980,0.205179,0.347830,0.298338,0.533275,0.017492,0.212962,0.348762,...,0.341159,0.538246,0.018980,0.235823,0.348122,0.354179,0.540306,0.021187,0.242840,0.347115
17,0.272226,0.533544,0.021187,0.198140,0.346623,0.284327,0.532804,0.018980,0.205176,0.347827,...,0.327299,0.536552,0.017493,0.228446,0.348891,0.341158,0.538245,0.018980,0.235820,0.348119


# TEST AVEC MINI DATASET DE TEST

In [74]:
# =========================================
# 1. Préparation du nouvel ensemble de test
# =========================================

# Tester avec un mini dataset de l'ensemble de Train, on est sensé d,avoir les metriques a 100%

pannes = [
    {'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'start': '2020-05-29 23:30:00', 'end': '2020-05-30 06:00:00'},
    {'start': '2020-06-05 10:00:00', 'end': '2020-06-07 14:30:00'},
    {'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
         ]

test2_periods  = [{'start': '2020-05-29 23:00:00', 'end': '2020-05-29 23:45:00'}]


test2_df = dd.concat([
    df_dask[(df_dask['timestamp'] >= period['start']) & (df_dask['timestamp'] <= period['end'])]
    for period in test2_periods
])

# Regrouper les partitions pour éviter les coupures temporelles
test2_df = test2_df.repartition(npartitions=1).reset_index(drop=True)

# Vérification du nombre de lignes et partitions
print(f"Nombre de lignes dans le test : {test2_df.shape[0].compute()}")
print(f"Nombre de partitions dans le test : {test2_df.npartitions}")

Nombre de lignes dans le test : 271
Nombre de partitions dans le test : 1


In [75]:
# =======================================================================
# 2. Génération des fenêtres glissantes pour l'ensemble de test avec Dask
# =======================================================================

# Démarrer le chronometre
start_time = time.time()

# Fonction pour créer des fenêtres de test
window_size = 6    # Taille de la fenêtre
step_size = 1      # Pas de glissement

def create_windows(df):
    windows = []
    df = df.compute()  # Convertir en Pandas pour traitement en mémoire
    for i in range(0, len(df) - window_size + 1, step_size):
        window_data = df.iloc[i:i + window_size][continuous_features].values.flatten()
        panne_value = df.iloc[i:i + window_size]["panne"].mode()[0]  # Classe majoritaire
        windows.append({
            "index": i,
            "timestamp": df.iloc[i]["timestamp"],
            "sensor_data": window_data.tolist(),
            "panne": panne_value
        })
    return windows

# Générer les fenêtres de test
print("Génération des fenêtres de test...")
test_windows = create_windows(test2_df)

print(f"Nombre de fenêtres générées : {len(test_windows)}")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

Génération des fenêtres de test...
Nombre de fenêtres générées : 266
Temps d'exécution: 6.30 secondes


In [78]:
# ====================================
# 7. Prédiction sur l'ensemble de test
# ====================================

# Démarrer le chronometre
start_time = time.time()

# Fonction de recherche des k plus proches voisins en utilisant la distance euclidienne
def find_similar_windows(test_vector, train_vectors, train_labels, k=5):
    # Calculer les distances entre le vecteur de test et les vecteurs d'entrainement
    distances = cdist([test_vector], train_vectors, metric="euclidean")
    #distances = cdist([test_vector], train_vectors, metric="cosine")
    
    # Trouver les indices des k plus proches voisins
    # nearest_indices = np.argsort(distances, axis=1)[:, :k]
    nearest_indices = np.argsort(distances, axis=1)[:, :k].astype(int)

    # Prédiction par vote majoritaire
    similar_pannes = train_labels[nearest_indices.flatten()]
    predicted_panne = np.bincount(similar_pannes).argmax()

    return predicted_panne

# Appliquer la prédiction sur l'ensemble de test
predictions = []
for test_sample in tqdm(test_windows, desc="Prédictions en cours"):
    test_vector = test_sample["sensor_data"]
    predicted_panne = find_similar_windows(test_vector, X_train_loaded, y_train_loaded)
    
    predictions.append({
        "timestamp": test_sample["timestamp"],
        "actual_panne": test_sample["panne"],
        "predicted_panne": predicted_panne
    })

# Convertir en DataFrame Pandas puis en Dask
predictions_df = pd.DataFrame(predictions)
predictions_dask = dd.from_pandas(predictions_df, npartitions=1)

# Sauvegarder les prédictions en format Parquet
# predictions_dask.to_parquet("Generated_Files/test_predictions.parquet")
# print("Prédictions sauvegardées avec succes.")

# Arrêter le chronometre
end_time = time.time()
print(f"Temps d'exécution: {end_time - start_time:.2f} secondes")

Prédictions en cours:   0%|                                                                    | 0/266 [00:02<?, ?it/s]


TypeError: only integer scalar arrays can be converted to a scalar index

In [60]:
# Sauvegarde au format CSV pour analyse ultérieure
#predictions_df.to_csv("Generated_Files/final2_predictions.csv", index=False)

#print("Les prédictions finales ont été enregistrées avec succes.")

Les prédictions finales ont été enregistrées avec succes.


In [61]:
# Création du tableau récapitulatif sans la colonne "Prédictions"
recap_data_corrected = {
    "Classe": [
        f"{int(class_label)} (Pas de panne)" if class_label == 0 else
        f"{int(class_label)} (En pleine panne)" if class_label == 1 else
        f"{int(class_label)} (Panne imminente)"
        for class_label in class_counts_real.index
    ],
    "Observations réelles": class_counts_real.values,
    "Prédictions correctes": correct_predictions.values,
    "Remarque": [
        "" if class_label in class_counts_pred.index else "Le modele n'a jamais prédit cette classe."
        for class_label in class_counts_real.index
    ]
}

# Création du DataFrame corrigé
recap_df_corrected = pd.DataFrame(recap_data_corrected)

# Affichage du tableau récapitulatif sans la colonne des prédictions incorrectes
recap_df_corrected


,Classe,Observations réelles,Prédictions correctes,Remarque
0,0 (Pas de panne),737454,737141,
1,1 (En pleine panne),1621,10,
2,2 (Panne imminente),179,0,Le modele n'a jamais prédit cette classe.
